# **Training Data**

In [6]:
!pip install pandas sentence-transformers pandarallel faiss-cpu

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [21]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
data = [['What is the capital of France?', 'geography'],
        ['Name the tallest mountain in the world', 'geography'],
        ['How does quantum computing work?', 'technology'],
        ['Best practices for cybersecurity', 'technology'],
        ['Explain the theory of relativity', 'science'],
        ['What are the planets in our solar system?', 'science'],
        ['Who discovered electricity?', 'science'],
        ['What is the largest desert in the world?', 'geography'],
        ['Difference between classical and quantum computing?', 'technology'],
        ['What is the circumference of the Earth?', 'geography']]
df = pd.DataFrame(data, columns=['text', 'category'])

In [29]:
df

,text,category,Text_Embedding
0,What is the capital of France?,geography,"[0.08204814, 0.036055543, -0.00389289, -0.0048..."
1,Name the tallest mountain in the world,geography,"[-0.0572189, 0.072823, -0.052668616, 0.0212674..."
2,How does quantum computing work?,technology,"[-0.017192911, -0.00088927505, -0.071710624, 0..."
3,Best practices for cybersecurity,technology,"[0.010249724, 0.014398895, -0.08371828, -0.076..."
4,Explain the theory of relativity,science,"[-0.011516238, 0.0058955015, -0.024227908, 0.0..."
5,What are the planets in our solar system?,science,"[0.009328208, -0.02318438, -0.03444735, 0.0201..."
6,Who discovered electricity?,science,"[-0.09973723, 0.11617121, 0.014935421, 0.11708..."
7,What is the largest desert in the world?,geography,"[0.11325711, 0.042495348, -0.052063648, 0.0603..."
8,Difference between classical and quantum compu...,technology,"[-0.060413375, -0.03392305, -0.04219346, 0.039..."
9,What is the circumference of the Earth?,geography,"[0.04952417, 0.039973862, -0.024602192, 0.0038..."


In [11]:
def generate_embeddings(query):
  embeddings = model.encode(query)
  return embeddings

In [12]:
# Generate embeddings for all the products descriptions - approx 3 min to complete

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:

# Generate Embeddings for all the products
df['Text_Embedding'] = df['text'].parallel_apply(generate_embeddings)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [14]:
df.head()

,text,category,Text_Embedding
0,What is the capital of France?,geography,"[0.08204814, 0.036055543, -0.00389289, -0.0048..."
1,Name the tallest mountain in the world,geography,"[-0.0572189, 0.072823, -0.052668616, 0.0212674..."
2,How does quantum computing work?,technology,"[-0.017192911, -0.00088927505, -0.071710624, 0..."
3,Best practices for cybersecurity,technology,"[0.010249724, 0.014398895, -0.08371828, -0.076..."
4,Explain the theory of relativity,science,"[-0.011516238, 0.0058955015, -0.024227908, 0.0..."


In [20]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(384)   # build the index

In [22]:
index.add(np.stack(df['Text_Embedding'].to_list()))                  # add vectors to the index

# **Test Data**

In [23]:
query="What is the largest ocean in the Planet Earth?"

In [24]:
vector_test=generate_embeddings(query)

In [27]:
k = 3                        # we want to see 3 nearest neighbors
Distance, Index_postion = index.search(vector_test.reshape(1, -1) , k) # sanity check

In [28]:
Distance, Index_postion

(array([[0.7779903, 0.9309093, 1.2147343]], dtype=float32), array([[7, 9, 5]]))

In [30]:
df['category'][Index_postion[0][0]]

'geography'

In [32]:
result_data=pd.DataFrame({"distance":Distance[0],"index_postions":Index_postion[0]})
result_data

,distance,index_postions
0,0.777990,7
1,0.930909,9
2,1.214734,5


In [33]:
merged_data=pd.merge(result_data,df,left_on="index_postions",right_index=True)
merged_data

,distance,index_postions,text,category,Text_Embedding
0,0.777990,7,What is the largest desert in the world?,geography,"[0.11325711, 0.042495348, -0.052063648, 0.0603..."
1,0.930909,9,What is the circumference of the Earth?,geography,"[0.04952417, 0.039973862, -0.024602192, 0.0038..."
2,1.214734,5,What are the planets in our solar system?,science,"[0.009328208, -0.02318438, -0.03444735, 0.0201..."
